To obtain an atomistic structure from coarse grained simulations we need a CG structure and a topology file for the atomistic system.

In [1]:
import running_rabbit as rr

In [5]:
# input
cg_structure = "2xUBQ_cg.pdb"
aa_topology  = "2xUBQ_aa.top"

# output
aa_structure_raw = "2xUBQ_aa_raw.gro"    # initial structure after running backward.py
aa_structure     = "2xUBQ_aa.pdb"        # final structure after energy min.
path_em          = "./em_after_backmap"

In [6]:
# HOTFIX, convert into package at some point?
backward = "python /home/andrejb/Software/Backward/backward.py"

In [7]:
# perform backmapping
!{backward} -f {cg_structure} -p {aa_topology} -o {aa_structure_raw} -to gromos -kick 0.05

Error reading alex to martini mapping for DLPC (file: /home/andrejb/Software/Backward/Mapping/lys.charmm36.map).
Error reading alex to martini mapping for DGPC (file: /home/andrejb/Software/Backward/Mapping/lys.gromos.map).
Error reading alex to martini mapping for DOPC.5 (file: /home/andrejb/Software/Backward/Mapping/lys.gromos.map).
Error reading alex to martini mapping for DALPC (file: /home/andrejb/Software/Backward/Mapping/lys.gromos.map).
Error reading alex to martini mapping for DOPC.4 (file: /home/andrejb/Software/Backward/Mapping/cl4.gromos.map).
Error reading alex to martini mapping for DLiPC (file: /home/andrejb/Software/Backward/Mapping/cl4.gromos.map).
Error reading alex to martini mapping for DPPC (file: /home/andrejb/Software/Backward/Mapping/cl4.gromos.map).
Error reading alex to martini mapping for DPG1 (file: /home/andrejb/Software/Backward/Mapping/val.charmm36.map).
Error reading gromos to martini mapping for MGDG (file: /home/andrejb/Software/Backward/Mapping/sep.gr

In [8]:
!gmx editconf -f {aa_structure_raw} -o 2xUBQ_aa_raw.pdb 

                 :-) GROMACS - gmx editconf, VERSION 5.1.2 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra   Sebastian Fritsch 
  Gerrit Groenhof   Christoph Junghans   Anca Hamuraru    Vincent Hindriksen
 Dimitrios Karkoulis    Peter Kasson        Jiri Kraus      Carsten Kutzner  
    Per Larsson      Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff 
   Erik Marklund      Teemu Murtola       Szilard Pall       Sander Pronk   
   Roland Schulz     Alexey Shvetsov     Michael Shirts     Alfons Sijbers  
   Peter Tieleman    Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala Universit

The atomistic structure after backmapping is highly unphysical and has to be energy minimized to obtain a proper structure. Therefore, we perform energy minimization in three steps:
1. Backbone position constrained, small em step, higher force threshold, no bond constraints 
2. Backbone position constrained, bigger em step, lower force threshold, no bond constraints
3. no position constraints, bigger em step, lower force threshold, bonds constrained with LINKS 

In [11]:
# note: add running rabbit to setup simulations, therefore prepare rr template for EQ of backmapped structures
rabbit              = rr.Rabbit(ff="gromos54a7", template_name="backmap_em")
rabbit.structure    = "./2xUBQ_aa_raw.pdb"
rabbit.topology     = aa_topology
rabbit.destination  = path_em
rabbit.run()

True

In [12]:
# run EM
!cd em_after_backmap && bash setup

In [13]:
# copy final structure
!cp {path_em}/eq5.pdb {aa_structure}

In [14]:
# note: some atoms are not in the first unit cell after mackmapping
# resolve by:
!gmx editconf -f {aa_structure} -o {aa_structure} -pbc yes

                 :-) GROMACS - gmx editconf, VERSION 5.1.2 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra   Sebastian Fritsch 
  Gerrit Groenhof   Christoph Junghans   Anca Hamuraru    Vincent Hindriksen
 Dimitrios Karkoulis    Peter Kasson        Jiri Kraus      Carsten Kutzner  
    Per Larsson      Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff 
   Erik Marklund      Teemu Murtola       Szilard Pall       Sander Pronk   
   Roland Schulz     Alexey Shvetsov     Michael Shirts     Alfons Sijbers  
   Peter Tieleman    Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala Universit